In [1]:
pip install streamlit streamlit-folium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import streamlit as st
import folium
from streamlit_folium import st_folium
import joblib
import numpy as np
import ee
import datetime

# --- Initialize Earth Engine ---
ee.Authenticate()
ee.Initialize(project='ringed-trail-454308-d2')

2025-06-21 12:56:36.727 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [3]:
import geopandas as gpd
import json
from shapely.geometry import shape

In [4]:
code='''
import joblib
import streamlit as st
import folium
from streamlit_folium import st_folium
import numpy as np
import geopandas as gpd
import datetime
import ee
from shapely.geometry import shape

# Initialize Earth Engine
ee.Initialize(project='ringed-trail-454308-d2')

# Load trained ensemble model and scaler
model = joblib.load("this_ensemble_model.pkl")
scaler = joblib.load("this_scaler.pkl")

# UI setup
st.set_page_config(layout="wide")
st.title("🧠 Deforestation Prediction Grid Map (Maharashtra Region)")

@st.cache_data
def load_grid():
    return gpd.read_file("Maharashtra_5x5km_Grid.geojson")

gdf = load_grid()
gdf["predicted"] = None

# Toggle prediction color overlay
enable_colors = st.checkbox("🟩🟥 Show prediction color (green: forest, red: deforested)", value=True)

# Center map around Maharashtra
m = folium.Map(location=[19.5, 74], zoom_start=7.5, control_scale=True)

# Add grid cells with dynamic styling
for _, row in gdf.iterrows():
    pred = row["predicted"]
    fill = "gray"
    if enable_colors and pred in [0, 1]:
        fill = "green" if pred == 0 else "red"

    folium.GeoJson(
        row["geometry"],
        style_function=lambda x, fill_color=fill: {
            "fillColor": fill_color,
            "color": "black",
            "weight": 0.5,
            "fillOpacity": 0.6 if enable_colors else 0.1
        }
    ).add_to(m)

st.markdown("### 📍 Click on a grid cell to get deforestation prediction")
map_data = st_folium(m, height=600, width=900)

if map_data.get("last_clicked"):
    click_point = shape({
        "type": "Point",
        "coordinates": [map_data["last_clicked"]["lng"], map_data["last_clicked"]["lat"]]
    })

    selected = gdf[gdf.geometry.contains(click_point)]
    if selected.empty:
        st.warning("⚠️ Click within a valid grid cell.")
    else:
        cell_idx = selected.index[0]
        cell = selected.iloc[0]
        lon, lat = cell.geometry.centroid.x, cell.geometry.centroid.y
        st.info(f"🧭 Selected Cell Center: ({lat:.4f}, {lon:.4f})")

        point = ee.Geometry.Point([lon, lat])
        try:
            end_date = datetime.datetime.now().date()
            start_date = end_date - datetime.timedelta(days=30)

            # NDVI & EVI
            mod13 = ee.ImageCollection("MODIS/061/MOD13Q1") \
                .filterBounds(point).filterDate(str(start_date), str(end_date)) \
                .select(["NDVI", "EVI"]).mean()
            ndvi = mod13.select("NDVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("NDVI").getInfo()
            evi = mod13.select("EVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("EVI").getInfo()

            # NDMI
            def compute_ndmi(img):
                return img.addBands(img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"))

            ndmi_img = ee.ImageCollection("MODIS/061/MOD09GA") \
                .filterBounds(point).filterDate(str(start_date), str(end_date)) \
                .map(compute_ndmi).select("NDMI").mean()
            ndmi = ndmi_img.reduceRegion(ee.Reducer.mean(), point, 500).get("NDMI").getInfo()

            # Precipitation
            precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
                .filterDate(str(start_date), str(end_date)) \
                .select("precipitation").sum().reduceRegion(ee.Reducer.mean(), point, 5000).get("precipitation").getInfo()

            # LST
            lst_img = ee.ImageCollection("MODIS/061/MOD11A1") \
                .filterDate(str(start_date), str(end_date)) \
                .select("LST_Day_1km").mean()
            lst = lst_img.multiply(0.02).subtract(273.15).reduceRegion(ee.Reducer.mean(), point, 1000).get("LST_Day_1km").getInfo()

            # Treecover (static or annual)
            treecover_img = ee.ImageCollection("MODIS/061/MOD44B") \
                .filterBounds(point).select("Percent_Tree_Cover").mean()
            treecover = treecover_img.reduceRegion(ee.Reducer.mean(), point, 250).get("Percent_Tree_Cover").getInfo()

            # Seasonal anomaly
            month = end_date.month
            # These would ideally come from precomputed monthly means
            lst_season_mean = 30  # placeholder
            precip_season_mean = 50  # placeholder
            lst_anomaly = lst - lst_season_mean
            precip_anomaly = precip - precip_season_mean

            features = np.array([[evi, ndmi, lst_anomaly, precip_anomaly, treecover]])
            scaled = scaler.transform(features)
            pred = model.predict(scaled)[0]
            label = "Deforested" if pred == 1 else "Forest"

            gdf.at[cell_idx, "predicted"] = pred

            st.success(f"✅ Prediction: {label}")
            st.markdown("📊 **Feature Summary**")
            st.table({
                "NDVI": [round(ndvi, 4)],
                "EVI": [round(evi, 4)],
                "NDMI": [round(ndmi, 4)],
                "LST (°C)": [round(lst, 2)],
                "LST Anomaly": [round(lst_anomaly, 2)],
                "Precipitation (mm)": [round(precip, 2)],
                "Precip Anomaly": [round(precip_anomaly, 2)],
                "Treecover": [round(treecover, 1)]
            })

        except Exception as e:
            st.error("❌ Error during prediction.")
            st.text(str(e))

'''

In [6]:
with open("app.py", "w", encoding="utf-8") as f:
    f.write(code)


In [7]:
!wget -q -O - ipv4.icanhazip.com


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501